# Cost of Living Data Processing

### context

We have a raw csv downloaded from kaggle at this link -> https://www.kaggle.com/datasets/mvieira101/global-cost-of-living

Need to understand what's inside the data and engineer 1 or 2 features to use as a proxy for cost of living of our end users if they were to move to that city

### Basic exploration

In [3]:
import pandas as pd

path = "../raw_data/Alternative_sources_country_level/cost-of-living_raw.csv"
df = pd.read_csv(path)
df.head()

,city,country,x1,x2,x3,x4,x5,x6,x7,x8,...,x47,x48,x49,x50,x51,x52,x53,x54,x55,data_quality
0,Seoul,South Korea,7.68,53.78,6.15,3.07,4.99,3.93,1.48,0.79,...,110.36,742.54,557.52,2669.12,1731.08,22067.70,10971.90,2689.62,3.47,1
1,Shanghai,China,5.69,39.86,5.69,1.14,4.27,3.98,0.53,0.33,...,123.51,1091.93,569.88,2952.70,1561.59,17746.11,9416.35,1419.87,5.03,1
2,Guangzhou,China,4.13,28.47,4.98,0.85,1.71,3.54,0.44,0.33,...,43.89,533.28,317.45,1242.24,688.05,12892.82,5427.45,1211.68,5.19,1
3,Mumbai,India,3.68,18.42,3.68,2.46,4.30,2.48,0.48,0.19,...,41.17,522.40,294.05,1411.12,699.80,6092.45,2777.51,640.81,7.96,1
4,Delhi,India,4.91,22.11,4.30,1.84,3.68,1.77,0.49,0.19,...,36.50,229.84,135.31,601.02,329.15,2506.73,1036.74,586.46,8.06,1


In [4]:
print(df.shape)
#we have around 5000 cities and 58 costs for various goods for each city

df["country"].unique().shape

(4956, 58)


(215,)

In [5]:
#numbeo added a feature to specify cities which they think more data might be needed. Let's check what happens if we filter by that
#how many cities have "low data quality"?
df_high_quality_only = df[df['data_quality'] == 1]
df_high_quality_only.shape
#we have 4033 cities with "more data needed", and only 923 that are considered to have "high data quality"

(923, 58)

In [6]:
df_high_quality_only["country"].unique().shape
#there's only 146 countries if we filter out the "low quality" data (according to numbeo)

(146,)

In [7]:
df.rename(columns={'x54': 'average_income'}, inplace=True)


### Narrowing down the data?

*We have lots of features but we only care to get a few simple indicators to estimate the cost of living of an expat / nomad in that country... so I'll only select a couple of features and use them from now on*

In [8]:
#for now we'll use all cities, even the ones with more data needed

#you can uncomment this line to only use the high quality data
#df = df[df['data_quality'] == 1].copy()

# STRATEGY TO HANDLE THIS DATASET PROPERLY:
1. select only the columnns we want to keep
2. handle missing values
3. compute a single feature that estimates cost of living for the app user

ALTHOUGH WE'LL FIRST DO SOME ROUGH FAST CALCULATIONS TO JUST HAVE SOMETHING


### Rough calculations (data quality not guaranteed)

In [10]:
# Define lifestyle assumptions:
# - person is a single expat that decided to move to the city, for a long-term stay
# - Housing: living in a 1 bed apartment in the city centre, pays average utilities
# - Eating out: 10 meals out per month at inexpensive restaurants
# - Groceries: see below



# Housing: Using the cost of a 1-bedroom apartment in the city centre (x48)
df['housing'] = df['x48'] + df['x36']

# Food: Meals out cost (10 meals per month)
df['food_eating_out'] = 10 * df['x1']

# Food: Groceries cost
df["food_groceries"] = (
    6   * df['x9']   +  # Milk
    4   * df['x10']  +  # Bread
    2   * df['x11']  +  # Rice
    2   * df['x12']  +  # Eggs
    1   * df['x13']  +  # Cheese
    2   * df['x14']  +  # Chicken
    1   * df['x15']  +  # Beef
    1.5 * df['x16']  +  # Apples
    1.5 * df['x17']  +  # Banana
    1.5 * df['x18']  +  # Oranges
    1.5 * df['x19']  +  # Tomato
    2   * df['x20']  +  # Potato
    1   * df['x21']  +  # Onion
    2   * df['x22']  +  # Lettuce
    20  * df['x23']  +  # Water
    1   * df['x24']  +  # Wine (optional)
    2   * df['x25']     # Beer (optional)
)

# Transportation: Monthly transport pass (x29)
df['transportation'] = df['x29']

# Other things like Internet: internet (x38)
df['other'] =  df['x38']

# Now, compute the overall estimated monthly cost of living
df['monthly_cost'] = (df['housing'] + df['food_eating_out'] +
                      df['food_groceries'] + df['transportation'] +
                      df['other'])

# Optional: view the resulting columns for each city
result = df[['city', 'country', 'monthly_cost',"average_income"]]
result.head(30)

,city,country,monthly_cost,average_income
0,Seoul,South Korea,1271.105,2689.62
1,Shanghai,China,1379.990,1419.87
2,Guangzhou,China,767.945,1211.68
3,Mumbai,India,677.305,640.81
4,Delhi,India,413.370,586.46
5,Dhaka,Bangladesh,298.570,280.73
6,Osaka,Japan,1142.625,2322.46
7,Jakarta,Indonesia,753.400,509.12
8,Shenzhen,China,1012.140,1572.22
9,Kinshasa,Congo,3213.805,400.00


In [20]:
country_level_cost_expense = result.groupby('country').agg(
    average_monthly_cost=('monthly_cost', 'mean'),
    average_income=('average_income', 'mean')
).reset_index()

In [21]:
country_level_cost_expense = country_level_cost_expense.dropna() # Remove null values

In [25]:
print(country_level_cost_expense.shape)
country_level_cost_expense.head(30)

(177, 3)


,country,average_monthly_cost,average_income
0,Afghanistan,937.715000,191.463333
1,Albania,462.650714,368.735000
2,Algeria,326.213182,262.671034
4,Andorra,1189.260000,2842.183333
5,Angola,677.375000,1428.770000
8,Argentina,453.923333,443.372941
9,Armenia,1264.980000,376.080000
10,Aruba,1391.225000,1509.460000
11,Australia,1792.683800,3521.600833
12,Austria,1405.239444,2255.341538


### Exploring the filtered dataset

In [11]:
#quick null value check

df.isnull().sum().sort_values(ascending=False)

x40             2400
x53             2303
x52             2227
x29             2166
x43             1681
x51             1512
x28             1506
x50             1480
x54             1432
x49             1431
x32             1396
x39             1375
x48             1363
x37             1201
x42             1132
x34             1058
x31             1022
x55             1006
x35              995
x30              873
x25              721
x26              655
x45              617
x33              588
x15              566
x14              558
x47              553
x12              507
x46              503
x22              501
x4               496
x27              490
x36              488
x44              479
x13              478
x41              475
x19              469
x24              462
x2               451
x7               445
x18              441
x5               440
x20              437
x21              433
x1               428
x10              413
x23              388
x17          